# Multi-Modal RAG with Image Captioning

### Multi-Modal RAG 구현 절차

**1.  문서 수집 및 텍스트+이미지 추출:**
PDF, HTML, PPT 등에서 텍스트와 함께 이미지 요소를 추출한다.
이미지의 위치, 캡션, 섹션 정보 등 메타데이터도 함께 저장한다.

**2. 이미지 캡션 생성 (Image Captioning):**
추출된 이미지를 비전 모델(예: BLIP, OFA, GPT-4V 등)을 활용하여 자동으로 설명을 생성한다.
예시: 입력 이미지: 막대 그래프
생성 캡션: “2023년과 2024년의 월별 매출을 비교한 막대 차트 입니다.”

**3. 텍스트와 이미지 설명 통합:**
생성된 이미지 캡션과 원본 텍스트를 하나의 인덱싱 대상으로 구성한다.
문단 또는 섹션 기준으로 텍스트 + 시각 정보 묶음을 생성한다.

**4. 임베딩 및 저장:**
텍스트와 이미지 설명을 텍스트 임베딩 모델로 벡터화한다.
벡터 DB(예: FAISS, Qdrant 등)에 저장하여 질의 시 검색 가능하도록 구성한다.

**5. 질의 처리 및 검색:**
사용자 질문을 벡터화하여, 텍스트 및 이미지 설명을 포함한 세그먼트에서 유사도 기반 검색한다.
시각적 정보 기반 질문도 대응 가능하다.

**6. 멀티모달 응답 생성:**
검색된 텍스트 + 이미지 설명 기반으로 LLM이 최종 응답 생성한다.
필요 시, 원본 이미지 URL, 차트 위치 등을 함께 출력할 수 있다.


## Setting Up the Environment

In [19]:
import io
import numpy as np
import json
import fitz
from PIL import Image
import base64
import re
import tempfile
import shutil

In [20]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Document Processing Functions

In [21]:
import os
import tempfile
import shutil
import fitz  # PyMuPDF

def extract_content_from_pdf(pdf_path, output_dir=None):
    """
    PDF 파일에서 텍스트와 이미지를 모두 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로
        output_dir (str, 선택): 이미지 저장 디렉토리 (없으면 임시 디렉토리 생성)

    Returns:
        Tuple[List[Dict], List[Dict]]: 텍스트 정보 리스트, 이미지 정보 리스트
    """
    # output_dir가 없으면 임시 디렉토리 생성
    temp_dir = None
    if output_dir is None:
        temp_dir = tempfile.mkdtemp()
        output_dir = temp_dir
    else:
        os.makedirs(output_dir, exist_ok=True)

    text_data = []     # 추출된 텍스트 저장 리스트
    image_paths = []   # 추출된 이미지 경로 및 메타데이터 저장 리스트

    print(f"{pdf_path}에서 콘텐츠 추출 중...")

    try:
        with fitz.open(pdf_path) as pdf_file:
            # 각 페이지 순회
            for page_number in range(len(pdf_file)):
                page = pdf_file[page_number]

                # 텍스트 추출
                text = page.get_text().strip()
                if text:
                    text_data.append({
                        "content": text,
                        "metadata": {
                            "source": pdf_path,
                            "page": page_number + 1,
                            "type": "text"
                        }
                    })

                # 이미지 추출
                image_list = page.get_images(full=True)
                for img_index, img in enumerate(image_list):
                    xref = img[0]  # 이미지의 XREF
                    base_image = pdf_file.extract_image(xref)

                    if base_image:
                        image_bytes = base_image["image"]
                        image_ext = base_image["ext"]

                        # 이미지 파일 저장
                        img_filename = f"page_{page_number+1}_img_{img_index+1}.{image_ext}"
                        img_path = os.path.join(output_dir, img_filename)

                        with open(img_path, "wb") as img_file:
                            img_file.write(image_bytes)

                        image_paths.append({
                            "path": img_path,
                            "metadata": {
                                "source": pdf_path,
                                "page": page_number + 1,
                                "image_index": img_index + 1,
                                "type": "image"
                            }
                        })

        print(f"텍스트 {len(text_data)}개, 이미지 {len(image_paths)}개 추출 완료")
        return text_data, image_paths

    except Exception as e:
        print(f"콘텐츠 추출 중 오류 발생: {e}")
        if temp_dir and os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
        raise

## Chunking Text Content

In [22]:
def chunk_text(text_data, chunk_size=1000, overlap=200):
    """
    텍스트 데이터를 오버랩을 포함한 청크로 분할합니다.

    Args:
        text_data (List[Dict]): PDF에서 추출된 텍스트 데이터 (텍스트 + 메타데이터 포함)
        chunk_size (int): 각 청크의 문자 수
        overlap (int): 청크 간 중첩 문자 수

    Returns:
        List[Dict]: 분할된 텍스트 청크 리스트 (각 청크에 메타데이터 포함)
    """
    chunked_data = []  # 청크 데이터를 저장할 리스트 초기화

    for item in text_data:
        text = item["content"]  # 텍스트 본문 추출
        metadata = item["metadata"]  # 해당 텍스트의 메타데이터 추출

        # 텍스트가 너무 짧으면 그대로 저장 (절반 이하일 경우)
        if len(text) < chunk_size / 2:
            chunked_data.append({
                "content": text,
                "metadata": metadata
            })
            continue

        # 지정된 크기와 오버랩에 따라 청크 생성
        chunks = []
        for i in range(0, len(text), chunk_size - overlap):
            chunk = text[i:i + chunk_size]  # 청크 추출
            if chunk:
                chunks.append(chunk)

        # 생성된 각 청크에 메타데이터 추가
        for i, chunk in enumerate(chunks):
            chunk_metadata = metadata.copy()  # 기존 메타데이터 복사
            chunk_metadata["chunk_index"] = i  # 청크 인덱스
            chunk_metadata["chunk_count"] = len(chunks)  # 전체 청크 수

            chunked_data.append({
                "content": chunk,
                "metadata": chunk_metadata
            })

    print(f"총 {len(chunked_data)}개의 텍스트 청크가 생성되었습니다.")
    return chunked_data

## Image Captioning with OpenAI Vision

In [23]:
import base64

def encode_image(image_path):
    """
    이미지 파일을 base64 문자열로 인코딩합니다.

    Args:
        image_path (str): 이미지 파일 경로

    Returns:
        str: base64로 인코딩된 이미지 문자열
    """
    # 이미지 파일을 바이너리 읽기 모드로 열기
    with open(image_path, "rb") as image_file:
        # 파일 내용을 읽고 base64로 인코딩
        encoded_image = base64.b64encode(image_file.read())
        # base64 바이트 데이터를 문자열로 디코딩하여 반환
        return encoded_image.decode('utf-8')

In [24]:
import os
from PIL import Image

def generate_image_caption(image_path):
    """
    OpenAI의 비전 기능을 사용하여 이미지 캡션을 생성합니다.
    
    Args:
        image_path (str): 이미지 파일 경로
        
    Returns:
        str: 생성된 이미지 설명 캡션
    """
    # 파일이 존재하는지 및 이미지인지 확인
    if not os.path.exists(image_path):
        return "오류: 이미지 파일을 찾을 수 없습니다."
    
    try:
        # 이미지를 열어 유효성 검사
        Image.open(image_path)
        
        # 이미지를 base64로 인코딩
        base64_image = encode_image(image_path)
        
        # 캡션 생성을 위한 API 요청 구성
        response = client.chat.completions.create(
            model="gpt-4o-mini",  
            messages=[
                {
                    "role": "system",
                    "content": "당신은 학술 논문 이미지 설명에 특화된 AI 어시스턴트입니다. "
                    "이미지의 핵심 정보를 포착하는 상세한 캡션을 작성하세요. "
                    "이미지에 차트, 표, 도해 등이 포함되어 있다면, 해당 내용과 목적을 명확하게 설명하세요. "
                    "생성된 캡션은 이후 사용자가 이 콘텐츠에 대해 질문할 때 검색 가능하도록 최적화되어야 합니다."
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "이 이미지를 학술적 관점에서 자세히 설명해 주세요:"},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=300
        )
        
        # 응답에서 캡션 추출
        caption = response.choices[0].message.content
        return caption
    
    except Exception as e:
        # 예외 발생 시 오류 메시지 반환
        return f"캡션 생성 중 오류 발생: {str(e)}"

In [25]:
def process_images(image_paths):
    """
    모든 이미지를 처리하여 캡션을 생성합니다.

    Args:
        image_paths (List[Dict]): 추출된 이미지들의 경로 및 메타데이터 리스트

    Returns:
        List[Dict]: 캡션이 포함된 이미지 정보 리스트
    """
    image_data = []  # 캡션 포함 이미지 데이터를 저장할 리스트 초기화

    print(f"{len(image_paths)}개의 이미지에 대해 캡션 생성 중...")  # 총 이미지 수 출력
    for i, img_item in enumerate(image_paths):
        print(f"{i+1}/{len(image_paths)} 번째 이미지 처리 중...")  # 현재 이미지 진행상황 출력
        img_path = img_item["path"]  # 이미지 경로 추출
        metadata = img_item["metadata"]  # 이미지 메타데이터 추출

        # 이미지에 대한 캡션 생성
        caption = generate_image_caption(img_path)

        # 캡션이 포함된 이미지 데이터 추가
        image_data.append({
            "content": caption,        # 생성된 캡션
            "metadata": metadata,      # 이미지 메타데이터
            "image_path": img_path     # 이미지 파일 경로
        })

    return image_data  # 최종 리스트 반환

## Simple Vector Store Implementation

In [26]:
class MultiModalVectorStore:
    """
    멀티모달 콘텐츠를 위한 간단한 벡터 저장소 구현입니다.
    """
    def __init__(self):
        # 벡터, 콘텐츠, 메타데이터를 저장할 리스트 초기화
        self.vectors = []
        self.contents = []
        self.metadata = []
    
    def add_item(self, content, embedding, metadata=None):
        """
        단일 항목을 벡터 저장소에 추가합니다.
        
        Args:
            content (str): 텍스트 또는 이미지 캡션 등 콘텐츠
            embedding (List[float]): 임베딩 벡터
            metadata (Dict, 선택): 추가 메타데이터
        """
        # 임베딩, 콘텐츠, 메타데이터를 각각의 리스트에 추가
        self.vectors.append(np.array(embedding))
        self.contents.append(content)
        self.metadata.append(metadata or {})
    
    def add_items(self, items, embeddings):
        """
        여러 항목을 벡터 저장소에 일괄 추가합니다.
        
        Args:
            items (List[Dict]): 콘텐츠와 메타데이터를 포함한 항목 리스트
            embeddings (List[List[float]]): 각 항목에 대한 임베딩 리스트
        """
        # 각 항목과 임베딩을 순회하며 저장소에 추가
        for item, embedding in zip(items, embeddings):
            self.add_item(
                content=item["content"],
                embedding=embedding,
                metadata=item.get("metadata", {})
            )
    
    def similarity_search(self, query_embedding, k=5):
        """
        쿼리 임베딩과 가장 유사한 항목들을 검색합니다.
        
        Args:
            query_embedding (List[float]): 쿼리 임베딩 벡터
            k (int): 반환할 유사 항목 수
        
        Returns:
            List[Dict]: 상위 k개의 유사 항목
        """
        # 저장된 벡터가 없으면 빈 리스트 반환
        if not self.vectors:
            return []
        
        # 쿼리 임베딩을 넘파이 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 계산하여 유사도 리스트 구성
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도 기준으로 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 추출 및 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "content": self.contents[idx],
                "metadata": self.metadata[idx],
                "similarity": float(score)  # JSON 직렬화를 위한 float 변환
            })
        
        return results

## Creating Embeddings

In [27]:
def create_embeddings(texts, model="text-embedding-3-small"):
    """
    주어진 텍스트 리스트에 대해 임베딩 벡터를 생성합니다.
    
    Args:
        texts (List[str]): 입력 텍스트 리스트
        model (str): 사용할 임베딩 모델 이름
        
    Returns:
        List[List[float]]: 생성된 임베딩 벡터 리스트
    """
    # 입력이 비어 있는 경우 빈 리스트 반환
    if not texts:
        return []
        
    # OpenAI 또는 HuggingFace API 제한을 고려하여 배치 단위 처리
    batch_size = 100
    all_embeddings = []  # 전체 임베딩을 저장할 리스트
    
    # 입력 텍스트를 배치 단위로 순회하며 처리
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]  # 현재 배치 추출
        
        # 현재 배치에 대해 임베딩 생성 요청
        response = client.embeddings.create(
            model=model,
            input=batch
        )
        
        # 응답에서 임베딩 벡터 추출
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 전체 임베딩 리스트에 추가
    
    # 모든 임베딩 반환
    return all_embeddings

## Complete Processing Pipeline

In [28]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    멀티모달 RAG 처리를 위한 문서를 처리합니다.
    
    Args:
        pdf_path (str): PDF 파일 경로
        chunk_size (int): 각 텍스트 청크의 길이 (문자 수 기준)
        chunk_overlap (int): 청크 간 중첩 길이 (문자 수 기준)

    Returns:
        Tuple[MultiModalVectorStore, Dict]: 벡터 저장소와 문서 처리 요약 정보
    """
    # 이미지 추출 저장 경로 생성
    image_dir = "extracted_images"
    os.makedirs(image_dir, exist_ok=True)
    
    # PDF에서 텍스트와 이미지 추출
    text_data, image_paths = extract_content_from_pdf(pdf_path, image_dir)
    
    # 추출된 텍스트를 청크 단위로 분할
    chunked_text = chunk_text(text_data, chunk_size, chunk_overlap)
    
    # 이미지에 대해 캡션 생성
    image_data = process_images(image_paths)
    
    # 텍스트 청크와 이미지 설명을 하나로 합침
    all_items = chunked_text + image_data
    
    # 임베딩 생성을 위한 텍스트만 추출
    contents = [item["content"] for item in all_items]
    
    # 임베딩 생성
    print("전체 콘텐츠에 대해 임베딩 생성 중...")
    embeddings = create_embeddings(contents)
    
    # 벡터 저장소 생성 및 항목 추가
    vector_store = MultiModalVectorStore()
    vector_store.add_items(all_items, embeddings)
    
    # 문서 정보 요약 (텍스트, 이미지 개수 등)
    doc_info = {
        "text_count": len(chunked_text),
        "image_count": len(image_data),
        "total_items": len(all_items),
    }
    
    # 요약 출력
    print(f"벡터 저장소에 총 {len(all_items)}개 항목 추가 완료 "
          f"({len(chunked_text)}개 텍스트 청크, {len(image_data)}개 이미지 캡션)")
    
    # 벡터 저장소와 문서 요약 정보 반환
    return vector_store, doc_info

## Query Processing and Response Generation

In [29]:
def query_multimodal_rag(query, vector_store, k=5):
    """
    멀티모달 RAG 시스템에 쿼리를 수행합니다.
    
    Args:
        query (str): 사용자 질문
        vector_store (MultiModalVectorStore): 문서 콘텐츠가 저장된 벡터 저장소
        k (int): 검색할 관련 항목 개수

    Returns:
        Dict: 쿼리 결과 및 생성된 응답
    """
    print(f"\n=== 쿼리 처리 중: {query} ===\n")
    
    # 쿼리에 대한 임베딩 생성
    query_embedding = create_embeddings(query)
    
    # 벡터 저장소에서 관련 콘텐츠 검색
    results = vector_store.similarity_search(query_embedding, k=k)
    
    # 검색 결과를 텍스트와 이미지로 구분
    text_results = [r for r in results if r["metadata"].get("type") == "text"]
    image_results = [r for r in results if r["metadata"].get("type") == "image"]
    
    print(f"관련 항목 {len(results)}개 검색됨 (텍스트 {len(text_results)}개, 이미지 캡션 {len(image_results)}개)")
    
    # 검색된 콘텐츠를 기반으로 AI 응답 생성
    response = generate_response(query, results)
    
    # 결과 딕셔너리로 정리하여 반환
    return {
        "query": query,
        "results": results,
        "response": response,
        "text_results_count": len(text_results),
        "image_results_count": len(image_results)
    }

In [30]:
def generate_response(query, results):
    """
    쿼리와 검색된 결과를 기반으로 응답을 생성합니다.
    
    Args:
        query (str): 사용자 질문
        results (List[Dict]): 검색된 텍스트 및 이미지 콘텐츠
        
    Returns:
        str: 생성된 응답 텍스트
    """
    # 검색된 결과들을 기반으로 문맥(context) 구성
    context = ""
    
    for i, result in enumerate(results):
        # 콘텐츠 유형 판단 (텍스트 또는 이미지 캡션)
        content_type = "Text" if result["metadata"].get("type") == "text" else "Image caption"
        # 페이지 번호 추출 (없으면 unknown)
        page_num = result["metadata"].get("page", "unknown")
        
        # 콘텐츠 유형 및 페이지 정보 추가
        context += f"[{content_type} from page {page_num}]\n"
        # 실제 콘텐츠 추가
        context += result["content"]
        context += "\n\n"
    
    # 시스템 프롬프트 (AI에게 역할 및 응답 방식 지시)
    system_message = """당신은 텍스트와 이미지를 포함한 문서를 기반으로 질문에 답변하는 AI 어시스턴트입니다. 
    당신은 문서로부터 검색된 텍스트와 이미지 캡션을 제공받았습니다. 
    이 정보를 사용하여 질문에 대해 정확하고 포괄적인 답변을 작성하십시오.
    만약 이미지나 도표에서 얻은 정보라면, 그 출처를 언급하십시오.
    검색된 정보만으로 질문에 완전히 답변할 수 없다면, 그 한계를 인정하십시오."""

    # 사용자 메시지: 질문 + 문맥
    user_message = f"""질문: {query}

    검색된 콘텐츠:
    {context}

    위 내용을 기반으로 질문에 답변해 주세요.
    """
    
    # LLM을 호출하여 응답 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        temperature=0.1  # 정확도 중심으로 창의성 최소화
    )
    
    # 생성된 응답 텍스트 반환
    return response.choices[0].message.content

## Evaluation Against Text-Only RAG

In [31]:
def build_text_only_store(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    비교용 텍스트 전용 벡터 저장소를 구축합니다.
    
    Args:
        pdf_path (str): PDF 파일 경로
        chunk_size (int): 각 청크의 문자 수
        chunk_overlap (int): 청크 간 중첩 문자 수

    Returns:
        MultiModalVectorStore: 텍스트 전용 벡터 저장소
    """
    # PDF에서 텍스트만 추출 (이미지 무시)
    text_data, _ = extract_content_from_pdf(pdf_path, None)
    
    # 텍스트를 청크 단위로 분할
    chunked_text = chunk_text(text_data, chunk_size, chunk_overlap)
    
    # 임베딩 생성을 위한 텍스트만 추출
    contents = [item["content"] for item in chunked_text]
    
    # 텍스트 임베딩 생성
    print("텍스트 전용 콘텐츠에 대해 임베딩 생성 중...")
    embeddings = create_embeddings(contents)
    
    # 벡터 저장소 생성 및 항목 추가
    vector_store = MultiModalVectorStore()
    vector_store.add_items(chunked_text, embeddings)
    
    print(f"텍스트 전용 벡터 저장소에 {len(chunked_text)}개 항목 추가 완료")
    return vector_store

In [32]:
def evaluate_multimodal_vs_textonly(pdf_path, test_queries, reference_answers=None):
    """
    멀티모달 RAG와 텍스트 전용 RAG를 비교 평가합니다.
    
    Args:
        pdf_path (str): 평가할 PDF 파일 경로
        test_queries (List[str]): 테스트 질문 리스트
        reference_answers (List[str], 선택): 기준 정답 리스트
        
    Returns:
        Dict: 각 쿼리에 대한 평가 결과 및 전체 분석
    """
    print("=== 멀티모달 RAG vs 텍스트 전용 RAG 평가 시작 ===\n")
    
    # 멀티모달 RAG용 문서 처리 (텍스트 + 이미지)
    print("\n멀티모달 RAG용 문서 처리 중...")
    mm_vector_store, mm_doc_info = process_document(pdf_path)
    
    # 텍스트 전용 RAG용 문서 처리
    print("\n텍스트 전용 RAG용 문서 처리 중...")
    text_vector_store = build_text_only_store(pdf_path)
    
    results = []  # 평가 결과 저장 리스트
    
    # 각 쿼리에 대해 평가 실행
    for i, query in enumerate(test_queries):
        print(f"\n\n=== 쿼리 {i+1} 평가 중: {query} ===")
        
        # 기준 정답이 있다면 참조
        reference = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]
        
        # 멀티모달 RAG 응답 생성
        print("\n멀티모달 RAG 실행 중...")
        mm_result = query_multimodal_rag(query, mm_vector_store)
        
        # 텍스트 전용 RAG 응답 생성
        print("\n텍스트 전용 RAG 실행 중...")
        text_result = query_multimodal_rag(query, text_vector_store)
        
        # 응답 비교 평가
        comparison = compare_responses(
            query,
            mm_result["response"],
            text_result["response"],
            reference
        )
        
        # 현재 쿼리 결과 저장
        results.append({
            "query": query,
            "multimodal_response": mm_result["response"],
            "textonly_response": text_result["response"],
            "multimodal_results": {
                "text_count": mm_result["text_results_count"],
                "image_count": mm_result["image_results_count"]
            },
            "reference_answer": reference,
            "comparison": comparison
        })
    
    # 전체 평가 분석 생성
    overall_analysis = generate_overall_analysis(results)
    
    return {
        "results": results,                     # 각 쿼리별 평가 결과
        "overall_analysis": overall_analysis,   # 총괄 분석
        "multimodal_doc_info": mm_doc_info      # 멀티모달 처리 문서 정보
    }

In [33]:
def compare_responses(query, mm_response, text_response, reference=None):
    """
    멀티모달 응답과 텍스트 전용 응답을 비교 평가합니다.
    
    Args:
        query (str): 사용자 쿼리
        mm_response (str): 멀티모달 RAG의 응답
        text_response (str): 텍스트 전용 RAG의 응답
        reference (str, 선택): 기준 정답 (있으면 정확성 비교 가능)

    Returns:
        str: 응답 비교 분석 결과
    """
    # 시스템 프롬프트: 평가자 역할 및 비교 기준 설명
    system_prompt = """당신은 두 가지 RAG 시스템을 비교 평가하는 전문가입니다:
    1. 멀티모달 RAG: 텍스트와 이미지 캡션 모두에서 검색
    2. 텍스트 전용 RAG: 텍스트에서만 검색

    다음 기준에 따라 어느 응답이 질문에 더 잘 답했는지 평가하세요:
    - 정확성과 올바름
    - 정보의 완전성
    - 쿼리와의 관련성
    - 이미지 요소에서 온 고유한 정보 (멀티모달의 경우)"""

    # 사용자 프롬프트: 쿼리와 두 응답을 포함
    user_prompt = f"""질문: {query}

    멀티모달 RAG 응답:
    {mm_response}

    텍스트 전용 RAG 응답:
    {text_response}
    """

    # 기준 정답이 있는 경우 포함
    if reference:
        user_prompt += f"""
    기준 정답:
    {reference}
    """

        user_prompt += """
    두 응답을 비교하고 어떤 응답이 질문에 더 잘 답변했는지, 그 이유를 설명하세요.
    멀티모달 응답이에 이미지에서 온 구체적인 정보가 있다면 명시하십시오.
    """

    # 평가 생성 (정확성 중시: temperature=0)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    return response.choices[0].message.content

In [34]:
def generate_overall_analysis(results):
    """
    멀티모달 RAG vs 텍스트 전용 RAG의 종합 분석을 생성합니다.
    
    Args:
        results (List[Dict]): 각 쿼리에 대한 평가 결과 리스트

    Returns:
        str: 종합 분석 결과 (텍스트)
    """
    # 시스템 프롬프트: 평가 기준 및 분석 방향 안내
    system_prompt = """당신은 RAG 시스템을 평가하는 전문가입니다. 여러 개의 테스트 쿼리를 기반으로 
    멀티모달 RAG(텍스트 + 이미지)와 텍스트 전용 RAG을 비교하여 종합 분석을 작성하세요.

    다음 항목에 중점을 두십시오:
    1. 멀티모달 RAG가 텍스트 전용보다 뛰어났던 쿼리 유형
    2. 이미지 정보를 포함했을 때의 구체적 이점
    3. 멀티모달 접근 방식의 단점 또는 제약 사항
    4. 각 접근 방식을 언제 사용하는 것이 적절한지에 대한 전반적인 권장사항"""

    # 쿼리별 요약 텍스트 생성
    evaluations_summary = ""
    for i, result in enumerate(results):
        evaluations_summary += f"쿼리 {i+1}: {result['query']}\n"
        evaluations_summary += f"멀티모달은 텍스트 {result['multimodal_results']['text_count']}개, 이미지 캡션 {result['multimodal_results']['image_count']}개를 검색함\n"
        evaluations_summary += f"응답 비교 요약: {result['comparison'][:200]}...\n\n"

    # 사용자 프롬프트: 평가 요약과 함께 전반적인 분석 요청
    user_prompt = f"""다음은 총 {len(results)}개의 쿼리에 대해 멀티모달 RAG과 텍스트 전용 RAG의 평가 요약입니다. 
    이 데이터를 기반으로 두 접근 방식의 성능을 비교 분석해 주세요:

    {evaluations_summary}

    이미지 정보가 응답 품질에 어떻게 기여했는지(또는 기여하지 못했는지)에 특히 주목하여 
    멀티모달 RAG의 상대적인 강점과 약점을 포함한 포괄적인 분석을 작성해 주세요."""

    # LLM을 이용하여 분석 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0  # 분석의 일관성과 정확성을 위해 창의성 최소화
    )
    
    return response.choices[0].message.content

## Evaluation on Multi-Modal RAG vs Text-Only RAG

In [36]:
# PDF 문서 경로 설정
pdf_path = "../../dataset/attention_is_all_you_need.pdf"

# 텍스트 및 시각적 정보를 함께 타겟팅하는 테스트 쿼리 정의
test_queries = [
    "Transformer (base model)의 BLEU 점수는 얼마인가요?",
]

# (선택) 평가를 위한 기준 정답 제공
reference_answers = [
    "Transformer (base model)은 WMT 2014 영어→독일어 번역 과제에서 BLEU 점수 27.3, 영어→프랑스어 과제에서 38.1을 달성합니다.",
]

# 멀티모달 RAG vs 텍스트 전용 RAG 평가 실행
evaluation_results = evaluate_multimodal_vs_textonly(
    pdf_path=pdf_path,
    test_queries=test_queries,
    reference_answers=reference_answers
)

# 전체 분석 결과 출력
print("\n***전체 분석 결과***\n")
print(evaluation_results["overall_analysis"])

=== 멀티모달 RAG vs 텍스트 전용 RAG 평가 시작 ===


멀티모달 RAG용 문서 처리 중...
dataset/attention_is_all_you_need.pdf에서 콘텐츠 추출 중...
텍스트 15개, 이미지 3개 추출 완료
총 56개의 텍스트 청크가 생성되었습니다.
3개의 이미지에 대해 캡션 생성 중...
1/3 번째 이미지 처리 중...
2/3 번째 이미지 처리 중...
3/3 번째 이미지 처리 중...
전체 콘텐츠에 대해 임베딩 생성 중...
벡터 저장소에 총 59개 항목 추가 완료 (56개 텍스트 청크, 3개 이미지 캡션)

텍스트 전용 RAG용 문서 처리 중...
dataset/attention_is_all_you_need.pdf에서 콘텐츠 추출 중...
텍스트 15개, 이미지 3개 추출 완료
총 56개의 텍스트 청크가 생성되었습니다.
텍스트 전용 콘텐츠에 대해 임베딩 생성 중...
텍스트 전용 벡터 저장소에 56개 항목 추가 완료


=== 쿼리 1 평가 중: Transformer (base model)의 BLEU 점수는 얼마인가요? ===

멀티모달 RAG 실행 중...

=== 쿼리 처리 중: Transformer (base model)의 BLEU 점수는 얼마인가요? ===



/tmp/ipykernel_3597273/4126536131.py:75: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "similarity": float(score)  # JSON 직렬화를 위한 float 변환


관련 항목 5개 검색됨 (텍스트 5개, 이미지 캡션 0개)

텍스트 전용 RAG 실행 중...

=== 쿼리 처리 중: Transformer (base model)의 BLEU 점수는 얼마인가요? ===

관련 항목 5개 검색됨 (텍스트 5개, 이미지 캡션 0개)

***전체 분석 결과***

### 멀티모달 RAG와 텍스트 전용 RAG 비교 분석

#### 1. 멀티모달 RAG가 텍스트 전용보다 뛰어났던 쿼리 유형
이번 쿼리에서는 "Transformer (base model)의 BLEU 점수는 얼마인가요?"라는 질문에 대해 멀티모달 RAG가 텍스트 5개를 검색했지만 이미지 캡션은 포함되지 않았습니다. 이 경우, 멀티모달 RAG의 성능이 텍스트 전용 RAG와 동등하게 나타났습니다. 그러나 일반적으로 멀티모달 RAG는 다음과 같은 쿼리 유형에서 더 뛰어난 성능을 보일 수 있습니다:
- **비주얼 정보가 중요한 쿼리**: 예를 들어, 특정 이미지의 내용을 설명하거나, 이미지와 관련된 데이터를 요구하는 경우.
- **복합적인 정보 요구**: 텍스트와 이미지가 함께 제공되어야 하는 경우, 예를 들어, 제품 리뷰에서 이미지와 텍스트가 함께 제공될 때.

#### 2. 이미지 정보를 포함했을 때의 구체적 이점
이번 쿼리에서는 이미지 정보가 포함되지 않았지만, 일반적으로 멀티모달 RAG의 이미지 정보가 응답 품질에 기여하는 방식은 다음과 같습니다:
- **시각적 맥락 제공**: 이미지가 텍스트의 의미를 보강하거나 명확히 할 수 있습니다. 예를 들어, 특정 제품의 이미지를 포함하면 해당 제품의 특성을 더 잘 이해할 수 있습니다.
- **정보의 다양성**: 이미지와 텍스트가 결합되어 다양한 형태의 정보를 제공함으로써 사용자가 더 풍부한 정보를 얻을 수 있습니다.
- **감정적 반응 유도**: 이미지가 포함되면 사용자의 감정적 반응을 유도할 수 있어, 정보의 수용성을 높일 수 있습니다.

#### 3. 멀티모달 접근 방식의 단점 또는 제약 사항
멀티모달 RAG의 단점은 다음과 같